In [ ]:
using DelimitedFiles
using Plots
using DSP
using Statistics

In [ ]:
times = readdlm("/tmp/times.txt", ',', Float64)[10:4096, :]/1e9
dt, smoothed = times[:, 1], times[:, 2]
dt = (dt .- 1/143.85).*1e6
;

In [ ]:
# Quantize by truncating towards the quantization value
qtrunc(f, q) = trunc(f/q - 1)*q + q
# Filter with a cheap IIR filter
lowpass = digitalfilter(Lowpass(1/128), Butterworth(2))
dt_filt0 = filt(lowpass, dt)
lowpass = digitalfilter(Lowpass(1/128), Butterworth(1))
dt_filt1 = dt
dt_filt1 = filt(lowpass, dt_filt1)
dt_filt1 = filt(lowpass, dt_filt1)
# dt_filt1 = filt(lowpass, dt_filt1)
p1 = plot([abs.(dt_filt0), abs.(dt_filt1)], title="error μs", label=["old" "new"])
p2 = plot([abs.(dt_filt0), abs.(dt_filt1)], yaxis=(scale=:log10), title="log error μs", label=["old" "new"])
plot(p1, size=(800, 200), layout=(2,1), xlim=(0, length(dt)), ylim=(0,6))

In [ ]:
arr = dt_filt1[1000:3000]
1/mean(arr), 1e6*std(arr)

In [ ]:
b, a = coefb(lowpass), coefa(lowpass)

In [ ]:
conv(b,b), conv(a,a)